# Manipulate atmospheric forcing data to simulate the OTC effect from the field experiment

date created: 2023-04-14
by Eva Lieungh, Yeliz Yilmaz

This notebook...

1. documents decisions made with the observational data
2. takes COSMO reanalysis data with noleap calendar and makes a modified version of the temperature forcing with 1 degree increased temperature in daytime
3. checks that it worked
4. re-zips the new data

Assuming logger placement/removal dates coincide well with OTC placement and removal dates, data from INCLINE_metadata_LoggerDates. csv, https://osf.io/5nmfe, gives these dates:

| Site  | placement  |  removal  |
|  ---- | --------   | --------- |
| Ulvehaugen, ALP1 | 2019-06-11, 2021-06-17 | 2020-09-22, 2021-09-29 |
| Lavisdalen, ALP2 | 2019-06-19, 2021-06-15 | 2020-09-29, 2021-09-28 |
| Gudmedalen, ALP3 | 2019-06-12, 2021-06-16 | 2020-09-30, 2021-09-28 |
| Skjellingahaugen, ALP4 | 2019-06-20, 2021-06-18 | 2020-09-28, 2021-09-27 |

For simplicity, use June 16 and Sept 28 as placement and removal dates for all sites and years. 

In [45]:
# import libraries
import os
import netCDF4 as nc
import xarray as xr  # NetCDF data handling
import pandas as pd
import numpy as np
import datetime as dt
import zipfile # for unzipping
import shutil # easiest whole-directory zipping
from pathlib import Path  # For easy path handling

Read in data. See other notebooks for downloading and unzipping data if necessary. The datmdata should be copied in from the ALP4_cosmorea_noleap directory to make sure longer simulations will work. Work with copies of these files, in a new ALP4_cosmorea_warmed directory, with while keeping the source folder as a backup if something goes wrong.

In [46]:
# set path to data folder with atmospheric forcing
cosmo_path = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/data/ALP4_cosmorea_warmed/datmdata"))
print("cosmo_path:", cosmo_path)

cosmo_path: C:\Users\evaler\OneDrive - Universitetet i Oslo\Eva\PHD\FATES_INCLINE\data\ALP4_cosmorea_warmed\datmdata


Print some information about the forcing data before deciding how the modification can be done

In [52]:
example_file = str(Path(cosmo_path + "/" + f"clm1pt_ALP4_1995-01.nc"))
with nc.Dataset(example_file, 'r') as ds:
    # List all variables in the file
    print("Variables:", ds.variables.keys())
    # print some attributes
    for attr_name in ds.ncattrs():
            print(f"  {attr_name}: {getattr(ds, attr_name)}")
    # Access the "time" variable
    time_var = ds.variables['time']
    # Print time info
    print("Time attributes:", time_var.ncattrs())
    print("Calendar:", time_var.calendar)
    print("Time units:", time_var.units)
    # Access the temperature variable and print the first values
    TBOT_var = ds.variables['TBOT']
    print("Temp. attributes:", TBOT_var.ncattrs())
    print("Temp. units", TBOT_var.long_name)
    print("Temp. units", TBOT_var.units)
    print("Values:", TBOT_var[0:10])

Variables: dict_keys(['EDGEW', 'EDGEE', 'EDGES', 'EDGEN', 'LONGXY', 'LATIXY', 'SWDIFDS_RAD', 'SWDIRS_RAD', 'RAIN_CON', 'RAIN_GSP', 'SNOW_GSP', 'SNOW_CON', 'PRECTmms', 'TBOT', 'WIND', 'PSRF', 'SHUM', 'FLDS', 'time'])
  creation_date: ti 21.2.2023 10.20.38 +0100
  history: Wed Apr 19 11:45:22 2023: ncatted -O -a calendar,time,o,c,noleap clm1pt_ALP4_1995-01.nc
Original data from COSMOREA6 data
  title: CLM single point datm input data
  conventions: CF-1.0
  case_title: COSMOREA6: SEEDCLIM
  NCO: netCDF Operators version 5.0.6 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)
Time attributes: ['standard_name', 'units', 'calendar', 'axis']
Calendar: noleap
Time units: hours since 1995-1-1 01:00:00
Temp. attributes: ['long_name', 'units', 'mode', '_FillValue', 'missing_value']
Temp. units temperature at the lowest atm level
Temp. units K
Values: [[[265.65213]]

 [[265.33612]]

 [[264.7357 ]]

 [[264.4444 ]]

 [[265.2419 ]]

 [[266.06653]]

 [[265.25058]]

 [[264.76358]]

 [[2

## Data modification

The data is provided at 3-hourly time steps. Let's choose 06:00 to 18:00 as 'daytime' when we apply the temperature modification. This should only be applied **between June 16 and Sept 28** each year. I will choose 1 degree (C or K) as a flat increase.

In the cosmo data, the time stamp (hour given, e.g. 01:00:00) is for the middle of the three hours that are averaged. So The three hours from 00:00:00-02:00:00 is set as 01:00:00. So I want time stamps **07:00:00** (06-08) and **16:00:00** (15-17; the alternative is 19:00:00 which goes from 18-20). 

The bash script below will...

1. loop through all f files in the current directory that end with -06, -07, -08, or -09.
2. make a copy of the original file (add "copy_" in front of old name)
3. subset the TBOT variable and store it as new files starting with "TBOT_" with cdo selname
4. subset times: For June, subset dates from the 16th, and September until the 28th. Subset daytime timestamps from the new TBOT files, saved as new files starting with "TBOT_seltime", using cdo seltime
5. add 1 degree kelvin to the new subsetted files using cdo addc (= add constant), saving new files starting with "TBOT_seltime_1deg"
6. define that when merging files, values with the same time stamps should be skipped for the first file provided in the command (export SKIP_SAME_TIME=1)
7. merge the modified TBOT- and time-subsetted files ("TBOT_seltime_1deg") back into the TBOT file
8. merge the new merged TBOT variable back into the original file, and name it the original normal file name.


In [ ]:
%%bash
pwd
cd ../data/ALP4_cosmorea_warmed/datmdata
pwd
for f in clm1pt_ALP4_*-06.nc; do cp $f copy_$f; cdo selname,TBOT copy_$f TBOT_$f; cdo selday,16/31 -seltime,07:00:00,10:00:00,13:00:00,16:00:00 TBOT_$f TBOT_seltime_$f; cdo addc,1 TBOT_seltime_$f TBOT_seltime_1deg_$f; export SKIP_SAME_TIME=1; cdo mergetime TBOT_seltime_1deg_$f TBOT_$f TBOT_merged_$f; cdo replace copy_$f TBOT_merged_$f $f; done
for f in clm1pt_ALP4_*-07.nc; do cp $f copy_$f; cdo selname,TBOT copy_$f TBOT_$f; cdo seltime,07:00:00,10:00:00,13:00:00,16:00:00 TBOT_$f TBOT_seltime_$f; cdo addc,1 TBOT_seltime_$f TBOT_seltime_1deg_$f; export SKIP_SAME_TIME=1; cdo mergetime TBOT_seltime_1deg_$f TBOT_$f TBOT_merged_$f; cdo replace copy_$f TBOT_merged_$f $f; done
for f in clm1pt_ALP4_*-08.nc; do cp $f copy_$f; cdo selname,TBOT copy_$f TBOT_$f; cdo seltime,07:00:00,10:00:00,13:00:00,16:00:00 TBOT_$f TBOT_seltime_$f; cdo addc,1 TBOT_seltime_$f TBOT_seltime_1deg_$f; export SKIP_SAME_TIME=1; cdo mergetime TBOT_seltime_1deg_$f TBOT_$f TBOT_merged_$f; cdo replace copy_$f TBOT_merged_$f $f; done
for f in clm1pt_ALP4_*-09.nc; do cp $f copy_$f; cdo selname,TBOT copy_$f TBOT_$f; cdo selday,1/28 -seltime,07:00:00,10:00:00,13:00:00,16:00:00 TBOT_$f TBOT_seltime_$f; cdo addc,1 TBOT_seltime_$f TBOT_seltime_1deg_$f; export SKIP_SAME_TIME=1; cdo mergetime TBOT_seltime_1deg_$f TBOT_$f TBOT_merged_$f; cdo replace copy_$f TBOT_merged_$f $f; done

The previous step created quite a few intermediary files that we can now remove:

In [63]:
%%bash
pwd
cd ../data/ALP4_cosmorea_warmed/datmdata
pwd
rm copy_* 
rm TBOT_*

/mnt/c/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/src
/mnt/c/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/data/ALP4_cosmorea_warmed/datmdata


### Unfinished python alternative

The code below is a now obsolete python version that sould do the same thing, but it is unfinished and not recommended to try without modifications and careful consideration. 

In [ ]:
# Load the .nc file
ds = xr.open_dataset(example_file)

# Extract the 'time' variable and convert it to a pandas DatetimeIndex
time = pd.to_datetime(ds['time'].data, format='%Y-%m-%d %H:%M:%S')

# Define the time range for daytime (06:00 to 18:00)
start_time = pd.Timestamp('1995-01-01 06:00:00')
end_time = pd.Timestamp('1995-01-01 18:00:00')

# Find the indices of time steps that fall within the daytime range (06:00 to 18:00)
daytime_indices = (ds['time'].dt.hour >= 6) & (ds['time'].dt.hour <= 18)
print(np.where(daytime_indices)[0])

# Use np.ix_ to create a 1D boolean mask for indexing the temperature variable
daytime_indices_expanded = np.ix_(daytime_indices)

# Add 1 degree to the temperature variable for the daytime range
ds['TBOT'].data[daytime_indices_expanded] += 1

# Save the modified data to a new .nc file
ds.to_netcdf(str(Path(cosmo_path + "/" + f"clm1pt_ALP4_1995-01.nc")))

### Check if everything looks correct

Print some info about an example file again to check whether it worked, without breaking anything

In [65]:
example_file = str(Path(cosmo_path + "/" + f"clm1pt_ALP4_1995-06.nc"))
with nc.Dataset(example_file, 'r') as ds:
    # List all variables in the file
    print("Variables:", ds.variables.keys())
    # print some attributes
    for attr_name in ds.ncattrs():
            print(f"  {attr_name}: {getattr(ds, attr_name)}")
    # Access the temperature variable and print the first values
    TBOT_var = ds.variables['TBOT']
    print("Temp. attributes:", TBOT_var.ncattrs())
    print("Temp. units", TBOT_var.long_name)
    print("Temp. units", TBOT_var.units)
    print("Values:", TBOT_var[0:10])

Variables: dict_keys(['time', 'EDGEW', 'EDGEE', 'EDGES', 'EDGEN', 'LONGXY', 'LATIXY', 'SWDIFDS_RAD', 'SWDIRS_RAD', 'RAIN_CON', 'RAIN_GSP', 'SNOW_GSP', 'SNOW_CON', 'PRECTmms', 'TBOT', 'WIND', 'PSRF', 'SHUM', 'FLDS'])
  CDI: Climate Data Interface version 2.0.4 (https://mpimet.mpg.de/cdi)
  Conventions: CF-1.6
  creation_date: ti 21.2.2023 10.38.05 +0100
  history: Thu Apr 20 10:22:16 2023: cdo replace copy_clm1pt_ALP4_1995-06.nc TBOT_merged_clm1pt_ALP4_1995-06.nc clm1pt_ALP4_1995-06.nc
Wed Apr 19 11:45:22 2023: ncatted -O -a calendar,time,o,c,noleap clm1pt_ALP4_1995-06.nc
Original data from COSMOREA6 data
  title: CLM single point datm input data
  conventions: CF-1.0
  case_title: COSMOREA6: SEEDCLIM
  NCO: netCDF Operators version 5.0.6 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)
  CDO: Climate Data Operators version 2.0.4 (https://mpimet.mpg.de/cdo)
Temp. attributes: ['long_name', 'units', '_FillValue', 'missing_value', 'mode']
Temp. units temperature at the lowe

Compare temperature for two days that should have been modified. There are 8 3hr time steps in a day, so June 16 should start at 8*16=128, and the four daytime timesteps on June 16th should be 131-134:

In [71]:
original_file = str(Path(f"C:/Users/evaler/OneDrive - Universitetet i Oslo/Eva/PHD/FATES_INCLINE/data/ALP4_cosmorea/datmdata/clm1pt_ALP4_1995-06.nc"))
warmed_file = str(Path(cosmo_path + "/" + f"clm1pt_ALP4_1995-06.nc"))
with nc.Dataset(original_file, 'r') as ds_1:
    print("A:", ds_1.variables['TBOT'][127:130])
    print("B:", ds_1.variables['TBOT'][131:134]) 
    print("C:", ds_1.variables['TBOT'][135:136]) 
with nc.Dataset(warmed_file, 'r') as ds_2:
    print("Should be identical to A:", ds_2.variables['TBOT'][127:130])
    print("Should be +1 to B:", ds_2.variables['TBOT'][131:134]) 
    print("Should be identical to C:", ds_2.variables['TBOT'][135:136])

A: [[[280.65457]]

 [[280.72372]]

 [[279.87286]]]
B: [[[282.24356]]

 [[283.5888 ]]

 [[282.15   ]]]
C: [[[279.16333]]]
Should be identical to A: [[[280.65457]]

 [[280.72372]]

 [[279.87286]]]
Should be +1 to B: [[[283.24356]]

 [[284.5888 ]]

 [[283.15   ]]]
Should be identical to C: [[[279.16333]]]


### Re-zip the newly modified data

In [72]:
shutil.make_archive(cosmo_path + "/ALP4_cosmorea_warmed",
                    'zip', cosmo_path + "/ALP4_cosmorea_warmed")
print("zipping complete")

zipping complete
